# Payment day

Según las investigaciones de Asad et al. (2020), Gorodetskaya et al. (2021) y Riabykh et al. (2022) los anticipos y fechas de pago son variables que influyen en la demanda de los ATM. Por lo que en este archivo se calcularán y agregarán estas variables.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../data/aggregated_data_cleaning_complete.csv')
data.head()

,fecha_transaccion,codigo_cajero,tipo_cajero,saldo_inicial,demanda,abastecimiento,saldo_final,Type,Weekday,Holiday Sequence,isYesterdayHoliday,isHoliday,isTomorrowHoliday,isYesterdayWeekday,isTomorrowWeekday,isWeekday
0,2023-06-02,6,B,644690.0,243020.0,0.0,401670.0,No Holiday,FRIDAY,WHH,False,True,True,True,False,True
1,2023-06-02,32,B,775480.0,265420.0,0.0,510060.0,No Holiday,FRIDAY,WHH,False,True,True,True,False,True
2,2023-06-02,116,B,668240.0,244340.0,0.0,423900.0,No Holiday,FRIDAY,WHH,False,True,True,True,False,True
3,2023-06-02,525,A,30110.0,8250.0,226470.0,248330.0,No Holiday,FRIDAY,WHH,False,True,True,True,False,True
4,2023-06-02,302,A,70720.0,11760.0,350460.0,409420.0,No Holiday,FRIDAY,WHH,False,True,True,True,False,True


In [3]:
data[['codigo_cajero', 'fecha_transaccion', 'isWeekday']].loc[
    (data["codigo_cajero"] == 1)
    ].head(20)

,codigo_cajero,fecha_transaccion,isWeekday
654,1,2023-06-02,True
1198,1,2023-06-03,False
1614,1,2023-06-04,False
2519,1,2023-06-05,True
3133,1,2023-06-06,True
3876,1,2023-06-07,True
4315,1,2023-06-08,True
5278,1,2023-06-09,True
5802,1,2023-06-10,False
6719,1,2023-06-11,False


## Cálculo de día de pago (Fin de mes)

En perú, generalmente, la fecha de pago se realiza el último día del mes, siempre y cuando, sea un día laborable (día de semana desde lunes hasta viernes) y no sea feriado. En caso se incumpla estos criterios, se calcula la fecha anterior más cercana que sí cumpla.

In [4]:
payment_data_end_month = data.copy(deep=True)

# Convertimos la columna 'fecha_transaccion' a tipo datetime
payment_data_end_month['fecha_transaccion'] = pd.to_datetime(payment_data_end_month['fecha_transaccion'])
def calculate_payment_day(group):
    # Obtenemos el último día del mes
    last_day_of_month = group['fecha_transaccion'].max()

    # Verificamos si el último día del mes existe en el grupo
    if not group[group['fecha_transaccion'] == last_day_of_month].empty:
        # Si el último día del mes es laborable y no es feriado, ese es el día de pago
        if group.loc[group['fecha_transaccion'] == last_day_of_month, 'isWeekday'].values[0] and not group.loc[group['fecha_transaccion'] == last_day_of_month, 'isHoliday'].values[0]:
            group['isPaymentDay'] = group['fecha_transaccion'] == last_day_of_month
        else:
            # Si no es laborable o es feriado, buscamos hacia atrás hasta encontrar un día laborable y que no sea feriado
            for i in range(1, len(group)):
                check_date = last_day_of_month - pd.Timedelta(days=i)
                if not group[group['fecha_transaccion'] == check_date].empty:
                    if group.loc[group['fecha_transaccion'] == check_date, 'isWeekday'].values[0] and not group.loc[group['fecha_transaccion'] == check_date, 'isHoliday'].values[0]:
                        group['isPaymentDay'] = group['fecha_transaccion'] == check_date
                        break
            else:
                group['isPaymentDay'] = False
    else:
        group['isPaymentDay'] = False

    return group

# Agrupamos por mes y ATM para aplicar la lógica de cálculo de día de pago
payment_data_end_month['isPaymentDay'] = False

payment_data_end_month = payment_data_end_month.groupby([payment_data_end_month['fecha_transaccion'].dt.to_period('M'), 'codigo_cajero']).apply(calculate_payment_day)

# Convertimos de nuevo 'fecha_transaccion' a tipo string si es necesario
payment_data_end_month['fecha_transaccion'] = payment_data_end_month['fecha_transaccion'].dt.strftime('%Y-%m-%d')

# Mostramos el dataframe resultante
payment_data_end_month = payment_data_end_month.reset_index(drop=True)

pd.set_option('display.max_rows', 500)

payment_data_end_month[['codigo_cajero', 'fecha_transaccion', 'isWeekday', 'isHoliday', 'isPaymentDay']].loc[
    (payment_data_end_month['codigo_cajero'] == 1) &
    (pd.to_datetime(payment_data_end_month['fecha_transaccion']).dt.day > 27)
    ].head(10)


C:\Users\edurz\AppData\Local\Temp\ipykernel_28056\2037597851.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  payment_data_end_month = payment_data_end_month.groupby([payment_data_end_month['fecha_transaccion'].dt.to_period('M'), 'codigo_cajero']).apply(calculate_payment_day)


,codigo_cajero,fecha_transaccion,isWeekday,isHoliday,isPaymentDay
26,1,2023-06-28,True,False,True
27,1,2023-06-29,True,True,False
28,1,2023-06-30,True,True,False
20327,1,2023-07-28,True,True,False
20328,1,2023-07-29,False,True,False
20329,1,2023-07-30,False,False,False
20330,1,2023-07-31,True,False,True
42027,1,2023-08-28,True,False,False
42028,1,2023-08-29,True,False,False
42029,1,2023-08-30,True,True,False


## Cálculo de día de pago (Quincena)

Ocasionalmente en Perú, los pagos se realizan en dos fechas, a quincena y a fin de mes, por lo que se replicará el funcionamiento del cálculo de dia de pago a fin de mes, pero en quincena.

In [5]:
payment_data_fortnight = data.copy(deep=True)

# Convertimos la columna 'fecha_transaccion' a tipo datetime
payment_data_fortnight['fecha_transaccion'] = pd.to_datetime(payment_data_fortnight['fecha_transaccion'])
def calculate_payment_day(group):
    # Obtenemos el primer día del mes para luego calcular el 15
    first_day_of_month = group['fecha_transaccion'].min().replace(day=1)
    
    # Calculamos el día 15 de ese mes
    fifteenth_day_of_month = first_day_of_month + pd.DateOffset(days=14)

    # Verificamos si el último día del mes existe en el grupo
    if not group[group['fecha_transaccion'] == fifteenth_day_of_month].empty:
        # Si el último día del mes es laborable y no es feriado, ese es el día de pago
        if group.loc[group['fecha_transaccion'] == fifteenth_day_of_month, 'isWeekday'].values[0] and not group.loc[group['fecha_transaccion'] == fifteenth_day_of_month, 'isHoliday'].values[0]:
            group['isPaymentDay'] = group['fecha_transaccion'] == fifteenth_day_of_month
        else:
            # Si no es laborable o es feriado, buscamos hacia atrás hasta encontrar un día laborable y que no sea feriado
            for i in range(1, len(group)):
                check_date = fifteenth_day_of_month - pd.Timedelta(days=i)
                if not group[group['fecha_transaccion'] == check_date].empty:
                    if group.loc[group['fecha_transaccion'] == check_date, 'isWeekday'].values[0] and not group.loc[group['fecha_transaccion'] == check_date, 'isHoliday'].values[0]:
                        group['isPaymentDay'] = group['fecha_transaccion'] == check_date
                        break
            else:
                group['isPaymentDay'] = False
    else:
        group['isPaymentDay'] = False

    return group

# Agrupamos por mes y ATM para aplicar la lógica de cálculo de día de pago
payment_data_fortnight['isPaymentDay'] = False

payment_data_fortnight = payment_data_fortnight.groupby([payment_data_fortnight['fecha_transaccion'].dt.to_period('M'), 'codigo_cajero']).apply(calculate_payment_day)

# Convertimos de nuevo 'fecha_transaccion' a tipo string si es necesario
payment_data_fortnight['fecha_transaccion'] = payment_data_fortnight['fecha_transaccion'].dt.strftime('%Y-%m-%d')

# Mostramos el dataframe resultante
payment_data_fortnight = payment_data_fortnight.reset_index(drop=True)

pd.set_option('display.max_rows', 500)

payment_data_fortnight[['codigo_cajero', 'fecha_transaccion', 'isWeekday', 'isHoliday', 'isPaymentDay']].loc[
    (payment_data_fortnight['codigo_cajero'] == 1) &
    (pd.to_datetime(payment_data_fortnight['fecha_transaccion']).dt.day > 12) &
    (pd.to_datetime(payment_data_fortnight['fecha_transaccion']).dt.day < 16)
    ].head(10)


C:\Users\edurz\AppData\Local\Temp\ipykernel_28056\1813405189.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  payment_data_fortnight = payment_data_fortnight.groupby([payment_data_fortnight['fecha_transaccion'].dt.to_period('M'), 'codigo_cajero']).apply(calculate_payment_day)


,codigo_cajero,fecha_transaccion,isWeekday,isHoliday,isPaymentDay
11,1,2023-06-13,True,False,False
12,1,2023-06-14,True,False,False
13,1,2023-06-15,True,False,True
20312,1,2023-07-13,True,False,False
20313,1,2023-07-14,True,False,True
20314,1,2023-07-15,False,False,False
42012,1,2023-08-13,False,False,False
42013,1,2023-08-14,True,False,False
42014,1,2023-08-15,True,False,True
63712,1,2023-09-13,True,False,False


## Calculando semana de pago

Asad et al. (2020) indica que la semana de pago también podría ser más precisa que un día en específico, como fue calculado anteriormente, por lo que para calcular la semana de pago, en base al último día del mes y quincena, se capturará los 3 días anteriores y los 3 días en adelante, todos ellos serán considerados como la semana de pago.

In [6]:
data[['codigo_cajero', 'fecha_transaccion']]

,codigo_cajero,fecha_transaccion
0,6,2023-06-02
1,32,2023-06-02
2,116,2023-06-02
3,525,2023-06-02
4,302,2023-06-02
...,...,...
247795,9,2024-05-20
247796,7,2024-05-20
247797,694,2024-05-20
247798,29,2024-05-20


In [7]:
payment_data_week = data.copy(deep=True)

# Convertimos 'fecha_transaccion' a datetime si no lo es
payment_data_week['fecha_transaccion'] = pd.to_datetime(payment_data_week['fecha_transaccion'])

# Función para calcular si una fecha cae dentro de la ventana de fin de mes o quincena
def is_payweek(transaction_date):
    # Calculamos el fin del mes de la fecha actual, independientemente del día
    start_of_month =  pd.Timestamp(transaction_date.year, transaction_date.month, 1) + pd.offsets.MonthBegin(0)
    end_of_month = pd.Timestamp(transaction_date.year, transaction_date.month, 1) + pd.offsets.MonthEnd(0)
    mid_month = transaction_date.replace(day=15)
    
    # Rango de 3 días antes y 3 días después para quincena y fin de mes, incluyendo días del mes siguiente
    payweek_days_start = pd.date_range(start=start_of_month, periods=3)
    payweek_days_end = pd.date_range(end=end_of_month, periods=3)
    payweek_days_mid = pd.date_range(mid_month - pd.DateOffset(days=3), mid_month + pd.DateOffset(days=3), freq='D')

    # Verificamos si la fecha de transacción está en cualquiera de los dos rangos
    return transaction_date in payweek_days_end or transaction_date in payweek_days_mid or transaction_date in payweek_days_start


# Creamos la nueva columna 'isPayweek' usando la función
payment_data_week['isPayweek'] = payment_data_week['fecha_transaccion'].apply(is_payweek)

payment_data_week[['codigo_cajero', 'fecha_transaccion', 'isWeekday', 'isHoliday', 'isPayweek']].loc[
    (payment_data_week['codigo_cajero'] == 1)
    ].head(10)


,codigo_cajero,fecha_transaccion,isWeekday,isHoliday,isPayweek
654,1,2023-06-02,True,True,True
1198,1,2023-06-03,False,False,True
1614,1,2023-06-04,False,False,False
2519,1,2023-06-05,True,False,False
3133,1,2023-06-06,True,False,False
3876,1,2023-06-07,True,True,False
4315,1,2023-06-08,True,False,False
5278,1,2023-06-09,True,False,False
5802,1,2023-06-10,False,False,False
6719,1,2023-06-11,False,False,False


## Combinando Pago a fin de mes, quincena y semana de pago

In [8]:
# Pago a fin de mes y quincena
temp_payment_data_end_month = payment_data_end_month['isPaymentDay']
temp_payment_data_fortnight = payment_data_fortnight['isPaymentDay']

agg_data = data.copy(deep = True)
agg_data['isPaymentDay'] = temp_payment_data_end_month | temp_payment_data_fortnight
agg_data[['codigo_cajero', 'isWeekday', 'isHoliday', 'fecha_transaccion', 'isPaymentDay']]

,codigo_cajero,isWeekday,isHoliday,fecha_transaccion,isPaymentDay
0,6,True,True,2023-06-02,False
1,32,True,True,2023-06-02,False
2,116,True,True,2023-06-02,False
3,525,True,True,2023-06-02,False
4,302,True,True,2023-06-02,False
...,...,...,...,...,...
247795,9,True,False,2024-05-20,False
247796,7,True,False,2024-05-20,False
247797,694,True,False,2024-05-20,False
247798,29,True,False,2024-05-20,False


In [9]:
# Agregando semana de pago
agg_data['isPayweek'] = payment_data_week['isPayweek']
agg_data[['codigo_cajero', 'isWeekday', 'isHoliday', 'fecha_transaccion', 'isPaymentDay', 'isPayweek']]

,codigo_cajero,isWeekday,isHoliday,fecha_transaccion,isPaymentDay,isPayweek
0,6,True,True,2023-06-02,False,True
1,32,True,True,2023-06-02,False,True
2,116,True,True,2023-06-02,False,True
3,525,True,True,2023-06-02,False,True
4,302,True,True,2023-06-02,False,True
...,...,...,...,...,...,...
247795,9,True,False,2024-05-20,False,False
247796,7,True,False,2024-05-20,False,False
247797,694,True,False,2024-05-20,False,False
247798,29,True,False,2024-05-20,False,False


In [10]:
agg_data.to_csv('../data/aggregated_data_payment.csv', index=False)